# GET THE K EIGENMODES USING ML.


1. Laplacian Comparison (Compute NetworkX Lap)


2. Use the 1st PC of the groups of Cell Types - H's paper

  (FI in a class is very close.) -> We can use only one predictor. -> more reasonable feature space to reduce OF.


3. 1st PC of the genes  vs  Timescale? various variables maybe.


4. Parcellation & FC(this is time consuming). 

  200 celltype dataset is the best - 17 classes. Get one feature within each class. Mean is also good.
  
  (424, 1) vector

Discuss with Ashish about the conjugate eigens.

In [1]:
import mat4py
import numpy as np
import pandas as pd
import scipy.io
import os 
import math
import scipy.io
import matplotlib.pyplot as plt
from random import sample
import networkx as nx
from scipy.stats import pearsonr

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, mean_squared_error

In [2]:
## 
## load .mat data in Python
## 1 use scipy.io.loadmat
## 2 use mat4py.loadmat

CellNames = scipy.io.loadmat('../data/CellType_Names.mat')
CellNames = CellNames['cellnames'] 


## we have 424 region of interests
## what is the 424*424 matrix?
Connectomes = scipy.io.loadmat('../data/Connectomes.mat')
Connectome_direct = Connectomes['C_dir']


'''

Region volumes, in a 424 vector, to get connectivity density, divide
% each row in connectomes by each entry in the vector to get density. Units
% are in 200 micron per vertex voxels.

'''

CellType_volumn = mat4py.loadmat('../data/Regional_Volumes.mat')
CellType_volumn = CellType_volumn['region_vols']
Celltype_volumn =np.array([np.array(xi) for xi in CellType_volumn])
print(Celltype_volumn.shape)

# Nomarlize by the entry

Connectome_direct_density = Connectome_direct/Celltype_volumn

CellNames = [CellNames[0][x][0] for x in list(range(25))]

labels = ['P:'+ x for x in CellNames] + ['R:'+ x for x in CellNames]

(424, 1)


In [3]:
# Import Tasic data, using the 25 cell type features

Cell_type_data = mat4py.loadmat('../data/Tasic_nG_606_TypeDensity.mat')
Cell_type = Cell_type_data['cell_type_density_nG606']
Celltype_mtx =np.array([np.array(xi) for xi in Cell_type])

# Important : normalizing via the columns

Celltype_mtx_norm =(Celltype_mtx-Celltype_mtx.min(axis=0)) / (Celltype_mtx.max(axis=0) - Celltype_mtx.min(axis=0) )

In [4]:
# self-connection should be 0.
for i in range(424):
    if Connectome_direct_density[i,i]!=0:
        Connectome_direct_density[i,i] = 0

In [5]:
# Use nx as a Graph analyzing tool.

df = pd.DataFrame(data=Connectome_direct_density, index=list(range(424)), columns=list(range(424)))

G = nx.from_numpy_matrix(Connectome_direct_density,create_using=nx.DiGraph, parallel_edges=True )
G.name = "Graph from pandas adjacency matrix"
print("Name: ", G.name)
print("Type: ", G.is_directed())
print("Nodes: ", G.number_of_nodes())
print("Edges: ", G.number_of_edges())

Name:  Graph from pandas adjacency matrix
Type:  True
Nodes:  424
Edges:  64376


In [6]:
eigenvector_centrality = [value[1] for _,value in enumerate(nx.eigenvector_centrality(G, weight='weight').items())]
closeness_centrality = [value[1] for _,value in enumerate(nx.closeness_centrality(G).items())] 
degree_centrality = [value[1] for _,value in enumerate(nx.degree_centrality(G).items())]

betweenness_centrality = [value[1] for _,value in enumerate(nx.betweenness_centrality(G, weight='weight').items())]
load_centrality  = [value[1] for _,value in enumerate(nx.load_centrality(G, weight='weight').items())]

In [6]:
X=pd.DataFrame(data=Celltype_mtx_norm,
          index=range(424),
        columns=CellNames
               )

In [8]:
# % Assume you have a N X N connectivity matrix C
# K = 10; % Number of eigenmodes of interest
# deg = sum(C, 1);
# N = size(C,1);
# L = eye(N) - diag(1./sqrt(deg))*C*diag(1./sqrt(deg));
# [V, D] = eig(L);
# evals = diag(D);
# % We are interested in only the top K modes
# emodes = V(:,1:K);
# evals = evals(1:K);
# % Now add below the code to correlate each cell type with each of the above eigenmodes
# [R,p] = corr(emodes, celltypes);

In [8]:
# Alternative: Non-normalized Laplacian.
C = Connectome_direct_density
K = 10  # Number of eigenmodes of interest
deg = np.sum(C, axis=0) 
# Check the way of defining row_deg and column_deg

N = C.shape[0]
L = np.diag(deg) - C
evals, emodes = np.linalg.eig(L)
evals

array([ 2.80765338e+03+0.j        ,  2.80763181e+03+0.j        ,
        1.74787140e+03+0.j        ,  1.75416456e+03+0.j        ,
        1.58640890e+03+0.j        ,  1.59586938e+03+0.j        ,
        1.45875640e+03+0.j        ,  1.44966626e+03+0.j        ,
        1.15380691e+03+0.j        ,  1.15374680e+03+0.j        ,
        9.76522667e+02+0.j        ,  9.75001823e+02+0.j        ,
        7.42346296e+02+0.j        ,  7.38888026e+02+0.j        ,
        6.96489317e+02+0.j        ,  6.90292116e+02+0.j        ,
        6.80439298e+02+0.j        ,  6.48630993e+02+0.j        ,
        6.45801223e+02+0.j        ,  6.45659358e+02+0.j        ,
        6.34824655e+02+0.j        ,  6.17577716e+02+0.j        ,
        6.16303042e+02+0.j        ,  5.95345755e+02+0.j        ,
        5.07945055e+02+0.j        ,  5.07939457e+02+0.j        ,
        4.88222513e+02+0.j        ,  4.80916118e+02+0.j        ,
        4.79351135e+02+0.j        ,  4.70649446e+02+0.j        ,
        4.67534501e+02+0.

In [10]:
np.diag(C).shape

(424,)

In [13]:
# Assume you have a N x N connectivity matrix C
# ChatGPT
C = Connectome_direct_density
K = 10  # Number of eigenmodes of interest

# Check the way of defining row_deg and column_deg
deg_column = np.sum(C, axis=0)
deg_row = np.sum(C, axis=1)
N = C.shape[0]
L = np.eye(N) - np.diag(1./np.sqrt(deg_row)) @ C @ np.diag(1./np.sqrt(deg_column))
evals, emodes = np.linalg.eig(L)
# We are interested in only the top K modes
# emodes = emodes[:, :K]
# evals = evals[:K]
# Now add below the code to correlate each cell type with each of the above eigenmodes
# R, p = np.corrcoef(emodes, celltypes)

In [ ]:
# singular value decomposition Later
# SVD - continue using the Us and ignore Vs
# That handles asymetric matrices.

In [14]:
# Make the matrix symmetric. 
C_symm = (C + np.transpose(C))/2

In [15]:
# Lap for this
# Use the built-in Laplacian function from NetworkX
Gsym = nx.Graph(C_symm)
Lsym = nx.laplacian_matrix(Gsym).toarray()

# https://networkx.org/documentation/stable/reference/generated/networkx.linalg.laplacianmatrix.laplacian_matrix.html

In [16]:
Lsym

array([[ 8.54635812e+01, -2.20751648e+00, -9.68397332e-01, ...,
        -1.60723333e-02,  0.00000000e+00,  0.00000000e+00],
       [-2.20751648e+00,  3.34935317e+02, -1.35922858e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.68397332e-01, -1.35922858e-01,  1.45056347e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-1.60723333e-02,  0.00000000e+00,  0.00000000e+00, ...,
         1.26272188e+02, -2.60099623e-01, -9.09826287e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.60099623e-01,  8.94931728e+01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.09826287e-01,  0.00000000e+00,  1.19769502e+02]])

In [17]:
evals_sym, emodes_sym = np.linalg.eig(Lsym)

In [18]:
evals_sym

array([ 1.46559816e+03,  1.43270745e+03,  1.05737104e+03,  9.41807795e+02,
        9.08498533e+02,  8.41191450e+02,  8.27823318e+02,  8.11810476e+02,
        7.71287276e+02,  7.09426811e+02,  6.87210291e+02,  6.79381520e+02,
        6.83689611e+02,  6.47571762e+02,  5.90031619e+02,  5.56429754e+02,
        5.19638057e+02,  4.94394775e+02,  4.88116692e+02,  4.87509114e+02,
        4.83767829e+02,  4.82364313e+02,  4.81250841e+02,  4.71225853e+02,
        4.60117163e+02,  4.55985412e+02,  4.50257793e+02,  4.40106122e+02,
        4.30010741e+02,  4.24900510e+02,  4.06160601e+02,  4.02241690e+02,
        3.93390783e+02,  3.89837671e+02,  3.87417669e+02,  3.79842777e+02,
        3.78304351e+02,  3.77646909e+02,  3.67545949e+02, -3.78212603e-13,
        3.55637812e+02,  3.52833681e+02,  3.52758854e+02,  3.51354524e+02,
        3.47939008e+02,  3.42298646e+02,  3.41668001e+02,  3.34751839e+02,
        3.34447360e+02,  3.29283318e+02,  3.26420581e+02,  3.25137687e+02,
        3.22256643e+02,  

### To Sort the eigenmodes using the eigen values (ascending)

### Use evals_sym, emodes_sym

In [12]:
evals_sorted = sorted(evals, key=lambda x:np.abs(x))
evals_sorted = np.abs(evals_sorted)

# Get the sorted indices of evals
sorted_indices = sorted(range(len(evals)), key=lambda i: np.abs(evals[i]))

# Use the sorted indices to sort emodes
emodes_sorted = [emodes[i] for i in sorted_indices]


In [13]:
k = 10

In [14]:
emodes_topk = np.real(emodes_sorted[:k])
evals_topk = evals_sorted[:k]

In [15]:
# Do the same thing, use the K smallest eigen values 
emodes_sorted[:k][0].shape

(424,)

In [16]:
emodes_topk[0]

array([ 1.88839343e-02,  8.64731444e-07, -2.59245877e-02, -2.59245877e-02,
       -2.63497839e-02,  2.78601265e-02,  2.01179304e-02,  2.16019412e-02,
       -5.42900356e-02,  3.91650278e-02, -1.69508809e-02,  8.90197073e-03,
       -4.88881959e-03, -1.85923576e-02, -1.85923576e-02,  4.07399306e-04,
        2.07552349e-02, -9.81643001e-03, -1.05783376e-04,  1.04675221e-03,
        1.09131321e-02,  1.09131321e-02,  2.02811827e-02, -1.71402303e-02,
       -8.95865339e-04, -1.89549851e-03, -2.15882468e-03, -2.15882468e-03,
        3.98599364e-02, -5.20799873e-04, -5.20799873e-04,  4.25061344e-03,
        4.04359999e-03,  4.04359999e-03, -6.18191297e-02, -1.12788886e-02,
       -1.12788886e-02,  4.13751544e-03,  4.13751544e-03,  3.42402067e-02,
        3.42402067e-02, -4.61817822e-03, -4.61817822e-03, -5.54530316e-03,
        1.18504912e-02,  1.18504912e-02,  7.90099532e-03,  7.90099532e-03,
        1.85784752e-02,  1.85784752e-02, -3.35817336e-02, -3.06518104e-03,
       -3.06518104e-03,  

In [17]:

# evals and emodes in decreasing in decreasing order
is_decreasing = all(evals[i] >= evals[i+1] for i in range(len(evals)-1))

# Print the result
if is_decreasing:
    print("evals are in decreasing order.")
else:
    print("evals are not in decreasing order.")


evals are not in decreasing order.


In [18]:
len(emodes_sorted)

424

In [25]:
emodes_sorted[0]

array([ 1.88839343e-02+0.j        ,  8.64731444e-07+0.j        ,
       -2.59245877e-02+0.00032127j, -2.59245877e-02-0.00032127j,
       -2.63497839e-02+0.j        ,  2.78601265e-02+0.j        ,
        2.01179304e-02+0.j        ,  2.16019412e-02+0.j        ,
       -5.42900356e-02+0.j        ,  3.91650278e-02+0.j        ,
       -1.69508809e-02+0.j        ,  8.90197073e-03+0.j        ,
       -4.88881959e-03+0.j        , -1.85923576e-02+0.00040256j,
       -1.85923576e-02-0.00040256j,  4.07399306e-04+0.j        ,
        2.07552349e-02+0.j        , -9.81643001e-03+0.j        ,
       -1.05783376e-04+0.j        ,  1.04675221e-03+0.j        ,
        1.09131321e-02+0.00719454j,  1.09131321e-02-0.00719454j,
        2.02811827e-02+0.j        , -1.71402303e-02+0.j        ,
       -8.95865339e-04+0.j        , -1.89549851e-03+0.j        ,
       -2.15882468e-03+0.01124147j, -2.15882468e-03-0.01124147j,
        3.98599364e-02+0.j        , -5.20799873e-04+0.00363612j,
       -5.20799873e-04-0.

In [22]:
np.power(emodes_sorted[0],2)
#.shape

array([ 3.56602973e-04+0.00000000e+00j,  7.47760471e-13+0.00000000e+00j,
        6.71981033e-04-1.66574626e-05j,  6.71981033e-04+1.66574626e-05j,
        6.94311109e-04-0.00000000e+00j,  7.76186646e-04+0.00000000e+00j,
        4.04731123e-04+0.00000000e+00j,  4.66643863e-04+0.00000000e+00j,
        2.94740796e-03-0.00000000e+00j,  1.53389940e-03+0.00000000e+00j,
        2.87332362e-04-0.00000000e+00j,  7.92450828e-05+0.00000000e+00j,
        2.39005570e-05-0.00000000e+00j,  3.45513710e-04-1.49689826e-05j,
        3.45513710e-04+1.49689826e-05j,  1.65974194e-07+0.00000000e+00j,
        4.30779775e-04+0.00000000e+00j,  9.63622981e-05-0.00000000e+00j,
        1.11901226e-08-0.00000000e+00j,  1.09569019e-06+0.00000000e+00j,
        6.73350675e-05+1.57029897e-04j,  6.73350675e-05-1.57029897e-04j,
        4.11326370e-04+0.00000000e+00j,  2.93787493e-04-0.00000000e+00j,
        8.02574706e-07-0.00000000e+00j,  3.59291459e-06-0.00000000e+00j,
       -1.21710179e-04-4.85367364e-05j, -1.21710179

In [130]:
# Coorelation

R, p = np.zeros((len(emodes_sorted), X.shape[1])), np.zeros((len(emodes_sorted), X.shape[1]))
for i in range(424):
    for j in range(X.shape[1]):
        R[i,j], p[i,j] = pearsonr(np.power(emodes_sorted[i],2), X.iloc[:,j])

print(R) #Coorelation Coefficients
print(p) # P-values


/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  


[[ 0.03710928 -0.03421302  0.00039994 ...  0.08912597 -0.0831323
  -0.0396263 ]
 [ 0.06330525 -0.06674291 -0.01331772 ...  0.00997273 -0.05895478
   0.02036238]
 [-0.03351087 -0.0216859  -0.10038948 ... -0.0512586  -0.01272757
  -0.01811528]
 ...
 [ 0.02964158  0.0176761   0.01085734 ... -0.00246144  0.05792
   0.01805378]
 [ 0.03526435 -0.03771355  0.01932781 ...  0.06491322 -0.02638885
   0.13267393]
 [-0.03083973  0.00019862  0.01562365 ... -0.04207877  0.02829706
  -0.00099855]]
[[0.44598029 0.48229563 0.99344871 ... 0.06673593 0.08732026 0.41572151]
 [0.19326124 0.17012777 0.78452183 ... 0.83776755 0.22573319 0.67587937]
 [0.49133479 0.65612159 0.03880339 ... 0.29231545 0.79385061 0.7099332 ]
 ...
 [0.5427304  0.71665936 0.82360326 ... 0.95969639 0.23399756 0.7108738 ]
 [0.46893121 0.4386041  0.69148085 ... 0.18216894 0.58790815 0.0062198 ]
 [0.52653459 0.99674641 0.74837673 ... 0.38743421 0.5611943  0.98364401]]


In [131]:
R[0]

array([ 0.03710928, -0.03421302,  0.00039994,  0.0167805 , -0.04781306,
        0.09839407,  0.05251919, -0.06622266,  0.08618199,  0.06681019,
        0.12130635,  0.04854599,  0.09456177,  0.04293755,  0.03121273,
        0.02419126, -0.0051189 , -0.07702383, -0.09626677,  0.05088953,
        0.01646132, -0.0669175 ,  0.08912597, -0.0831323 , -0.0396263 ])

In [80]:
abs_R = np.abs(R)  # take absolute values of R
top_features_idx = abs_R.mean(axis=0).argsort()[::-1][:10]  # indices of top 10 features

# get the names of the top 10 features
top10_features = X.columns[top_features_idx].tolist()

In [81]:
top10_features

['Vip',
 'L5_PT',
 'L6b',
 'NP',
 'Macro',
 'Sst',
 'Sncg',
 'Chrna6',
 'Serpinf1',
 'Gad2']

In [82]:
#FUnction to Report top three features

def get_top_features(model, n=5):
    feature_importances = model.feature_importances_
    feature_importances_dict = {}
    for i, importance in enumerate(feature_importances):
        feature_importances_dict[i] = importance
    top_features = sorted(feature_importances_dict, key=feature_importances_dict.get, reverse=True)[:n]
    return top_features


In [86]:
emodes_topk[0].shape

(424,)

In [94]:
X.columns = X.columns.astype(str)


In [115]:
models = {
        'Linear Regression': LinearRegression(),
#           'Lasso Regression': Lasso(),
#           'Decision Tree Regression': DecisionTreeRegressor(),
          'Random Forest Regression': RandomForestRegressor(max_depth=3, bootstrap=True),
          'Gradient Boosting Regression': GradientBoostingRegressor(max_depth=3),
}

top_table = []
# This i means the ith Eigen Value
for i in range(10):
    y_i = emodes_topk[i]
    print(f"\n Now showing the most important cell type for {i}th eigenmode")
    
    # Itertae through the models
    for name, model in models.items():
        model.fit(X, y_i)
        y_i_pred = model.predict(X)
        r2 = r2_score(y_i, y_i_pred)
        mae = mean_absolute_error(y_i, y_i_pred)
        print(f"{name}:\n R-Squared = {r2:.4f}\n MeanAbsoluteError = {mae:.4f}\n")
        if hasattr(model,"feature_importances_"):
            # We can change the number of features to be displayed here.
            top_features = get_top_features(model, 6)
            print(f"\nTop X Features: {[X.columns[j] for j in top_features]}")
            top_table.append([X.columns[j] for j in top_features])
            print()
        


 Now showing the most important cell type for 0th eigenmode
Linear Regression:
 R-Squared = 0.0398
 MeanAbsoluteError = 0.0215

Random Forest Regression:
 R-Squared = 0.1655
 MeanAbsoluteError = 0.0204


Top Three Features: ['L5_IT', 'CR', 'Sst', 'NP', 'Macro', 'Oligo']

Gradient Boosting Regression:
 R-Squared = 0.6901
 MeanAbsoluteError = 0.0125


Top Three Features: ['CR', 'Sst', 'Macro', 'Oligo', 'L2_3_IT', 'Serpinf1']


 Now showing the most important cell type for 1th eigenmode
Linear Regression:
 R-Squared = 0.0987
 MeanAbsoluteError = 0.0196

Random Forest Regression:
 R-Squared = 0.2083
 MeanAbsoluteError = 0.0183


Top Three Features: ['Astro', 'Sst', 'Vip', 'Chrna6', 'L6b', 'Endo']

Gradient Boosting Regression:
 R-Squared = 0.7331
 MeanAbsoluteError = 0.0110


Top Three Features: ['L6b', 'Astro', 'Oligo', 'Sst', 'Chrna6', 'Serpinf1']


 Now showing the most important cell type for 2th eigenmode
Linear Regression:
 R-Squared = 0.0705
 MeanAbsoluteError = 0.0104

Random Fore

In [116]:
top_table

[['L5_IT', 'CR', 'Sst', 'NP', 'Macro', 'Oligo'],
 ['CR', 'Sst', 'Macro', 'Oligo', 'L2_3_IT', 'Serpinf1'],
 ['Astro', 'Sst', 'Vip', 'Chrna6', 'L6b', 'Endo'],
 ['L6b', 'Astro', 'Oligo', 'Sst', 'Chrna6', 'Serpinf1'],
 ['Endo', 'Slc17a6', 'NP', 'Sncg', 'Chrna6', 'L6b'],
 ['Endo', 'NP', 'Slc17a6', 'Macro', 'Pvalb', 'Sncg'],
 ['Astro', 'Macro', 'Sncg', 'CR', 'Oligo', 'NP'],
 ['CR', 'Astro', 'Sncg', 'Macro', 'Slc17a7', 'Chrna6'],
 ['Meis2', 'Astro', 'Endo', 'Sncg', 'NP', 'L2_3_IT'],
 ['Astro', 'NP', 'Chrna6', 'Endo', 'Pvalb', 'Lamp5'],
 ['CR', 'L5_IT', 'Sncg', 'Astro', 'Vip', 'Slc17a6'],
 ['Macro', 'LGv', 'Sncg', 'CR', 'L5_IT', 'Meis2'],
 ['Sst', 'Meis2', 'Astro', 'Vip', 'Sncg', 'Oligo'],
 ['Sst', 'Astro', 'Macro', 'Endo', 'Chrna6', 'NP'],
 ['Macro', 'NP', 'Slc17a6', 'Chrna6', 'Meis2', 'Endo'],
 ['Macro', 'NP', 'Astro', 'Meis2', 'Vip', 'Oligo'],
 ['Astro', 'CR', 'Vip', 'L2_3_IT', 'NP', 'Macro'],
 ['Astro', 'Macro', 'Endo', 'Vip', 'Oligo', 'Pvalb'],
 ['Sncg', 'Slc17a6', 'L2_3_IT', 'NP', 'Sst',

In [118]:
# Random Forest results for Laplacian Eigenmodes. 
# We choose RF because the r-squared is high but not 1, indicating less over-fitting than GBR.
gb_fi = top_table[1::2]
rf_fi = top_table[::2]

### The table above shows the top5 important cell types in predicting the ith eigenmode.

In [ ]:
# cell types vs emodes
# R, p  each emode and ctype

In [14]:
X

,Lamp5,Meis2,Pvalb,Serpinf1,Sncg,Sst,Vip,CR,L2_3_IT,L4,...,NP,Chrna6,Gad2,LGv,Slc17a6,Slc17a7,Astro,Macro,Oligo,Endo
0,0.101805,0.222524,0.302614,0.163599,0.122002,0.535885,0.170548,0.052602,0.000924,0.031791,...,0.014566,0.293695,0.390926,0.119970,0.054485,0.080485,0.205020,0.153997,0.220310,0.023048
1,0.433535,0.065233,0.057713,0.232302,0.620823,0.351885,0.272972,0.031347,0.226994,0.000000,...,0.551888,0.253789,0.031843,0.163988,0.000000,0.042441,0.320721,0.338157,0.060833,0.073400
2,0.552313,0.080393,0.019392,0.232100,0.188465,0.626230,0.528422,0.068763,0.293348,0.031445,...,0.425850,0.276182,0.063267,0.313050,0.003337,0.105231,0.459888,0.287689,0.047535,0.042527
3,0.250333,0.124450,0.018548,0.083036,0.091009,0.667059,0.267902,0.016532,0.081395,0.000000,...,0.140910,0.128628,0.490576,0.388156,0.001695,0.015331,0.312148,0.267270,0.177148,0.036023
4,0.318837,0.050211,0.021294,0.116680,0.162505,0.074857,0.388130,0.055558,0.065079,0.103319,...,0.117359,0.133240,0.104529,0.253774,0.027823,0.010218,0.080132,0.149537,0.100476,0.036720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,0.002916,0.000000,0.022791,0.013974,0.002849,0.002823,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055845,0.069322,0.064390,0.727708,0.000000,0.062062,0.094632,0.215801,0.154713
420,0.005589,0.000000,0.005976,0.014858,0.013354,0.095823,0.003662,0.000439,0.000000,0.000000,...,0.046640,0.068762,0.361866,0.061606,0.400586,0.000000,0.129010,0.154733,0.367528,0.090773
421,0.008294,0.014807,0.013935,0.051451,0.001822,0.000654,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.126891,0.021096,0.044091,0.729088,0.000000,0.075835,0.068349,0.340658,0.172590
422,0.000000,0.002242,0.000000,0.003021,0.000000,0.001471,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.014000,0.000073,0.000000,0.998757,0.000000,0.000000,0.025494,0.058646,0.249453


In [119]:
# Real part of emodes
for i in range(10):
    linear_model = LinearRegression(fit_intercept=True)
    print(f"{i}th eigenmode")
    X_i = X[gb_fi[i]]
    y_i = emodes_topk[i]
    
    linear_model.fit(X_i, y_i)
    y_i_pred = linear_model.predict(X_i)
    r2 = r2_score(y_i, y_i_pred)
    mae = mean_absolute_error(y_i, y_i_pred)
    print(f"R-Squared = {r2:.4f}\n MeanAbsoluteError = {mae:.4f}\n")

0th eigenmode
R-Squared = 0.0161
 MeanAbsoluteError = 0.0218

1th eigenmode
R-Squared = 0.0398
 MeanAbsoluteError = 0.0200

2th eigenmode
R-Squared = 0.0297
 MeanAbsoluteError = 0.0106

3th eigenmode
R-Squared = 0.0370
 MeanAbsoluteError = 0.0372

4th eigenmode
R-Squared = 0.0162
 MeanAbsoluteError = 0.0115

5th eigenmode
R-Squared = 0.0090
 MeanAbsoluteError = 0.0338

6th eigenmode
R-Squared = 0.0196
 MeanAbsoluteError = 0.0273

7th eigenmode
R-Squared = 0.0153
 MeanAbsoluteError = 0.0122

8th eigenmode
R-Squared = 0.0079
 MeanAbsoluteError = 0.0342

9th eigenmode
R-Squared = 0.0439
 MeanAbsoluteError = 0.0304



In [111]:
for i in range(10):
    linear_model = LinearRegression(fit_intercept=True)
    print(f"{i}th eigenmode")
    X_i = X[gb_fi[i]]
    y_i = emodes_topk[i]
    
    linear_model.fit(X_i, y_i)
    y_i_pred = linear_model.predict(X_i)
    r2 = r2_score(y_i, y_i_pred)
    mae = mean_absolute_error(y_i, y_i_pred)
    print(f"R-Squared = {r2:.4f}\n MeanAbsoluteError = {mae:.4f}\n")

0th eigenmode
R-Squared = 0.0309
 MeanAbsoluteError = 0.0175

1th eigenmode
R-Squared = 0.0213
 MeanAbsoluteError = 0.0151

2th eigenmode
R-Squared = 0.0745
 MeanAbsoluteError = 0.0074

3th eigenmode
R-Squared = 0.0237
 MeanAbsoluteError = 0.0280

4th eigenmode
R-Squared = 0.0336
 MeanAbsoluteError = 0.0098

5th eigenmode
R-Squared = 0.0475
 MeanAbsoluteError = 0.0287

6th eigenmode
R-Squared = 0.0538
 MeanAbsoluteError = 0.0171

7th eigenmode
R-Squared = 0.0611
 MeanAbsoluteError = 0.0092

8th eigenmode
R-Squared = 0.0195
 MeanAbsoluteError = 0.0296

9th eigenmode
R-Squared = 0.0312
 MeanAbsoluteError = 0.0199



In [106]:
for i in range(10):
    linear_model = LinearRegression(fit_intercept=False)
    print(f"{i}th eigenmode")
    X_i = X
    y_i = emodes_topk[i]
    
    linear_model.fit(X_i, y_i)
    y_i_pred = linear_model.predict(X_i)
    r2 = r2_score(y_i, y_i_pred)
    mae = mean_absolute_error(y_i, y_i_pred)
    print(f"{name}:\n R-Squared = {r2:.4f}\n MeanAbsoluteError = {mae:.4f}\n")


0th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0963
 MeanAbsoluteError = 0.0167

1th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0579
 MeanAbsoluteError = 0.0148

2th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.1283
 MeanAbsoluteError = 0.0071

3th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0617
 MeanAbsoluteError = 0.0274

4th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0735
 MeanAbsoluteError = 0.0096

5th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0615
 MeanAbsoluteError = 0.0280

6th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0613
 MeanAbsoluteError = 0.0167

7th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.1048
 MeanAbsoluteError = 0.0090

8th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.0347
 MeanAbsoluteError = 0.0292

9th eigenmode
Gradient Boosting Regression:
 R-Squared = 0.1208
 MeanAbsoluteError = 0.0189



In [26]:
import statsmodels.api as sm

In [110]:
for i in range(10):
    print(i)
    paralist = top_table[i]
    Y_s = emodes_topk[i]
    X_s = X[gb_fi[i]]
    X_s = sm.add_constant(X_s)
    model_s = sm.OLS(Y_s,X_s)
    results_s = model_s.fit()
    # results_s.params 
    print(results_s.summary())

0
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     2.215
Date:                Wed, 26 Apr 2023   Prob (F-statistic):             0.0408
Time:                        14:35:48   Log-Likelihood:                 1023.6
No. Observations:                 424   AIC:                            -2033.
Df Residuals:                     417   BIC:                            -2005.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0303      0.003     11.164      0

In [24]:
results_s.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.484
Date:                Wed, 26 Apr 2023   Prob (F-statistic):              0.194
Time:                        10:23:47   Log-Likelihood:                 685.06
No. Observations:                 424   AIC:                            -1358.
Df Residuals:                     418   BIC:                            -1334.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0029      0.003     -0.879      0.380      -0.009       0.004
L6_IT         -0.0471      0.025     -1.901      0.058      -0.096       0.002
L6_CT          0.0173      0.014      1.201      0.230      -0.011       0.046
Meis2         -0.0026      0.029     -0.091      0.927      -0.059       0.053
Lamp5         -0.0137      0.015     -0.911      0.363      -0.043       0.016
Serpinf1       0.0294      0.020      1.461      0.145      -0.010       0.069
==============================================================================
Omnibus:                      420.244   Durbin-Watson:                   2.152
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           107392.831
Skew:                           3.519   Prob(JB):                         0.00
Kurtosis:                      80.649   Cond. No.                         13.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Use the top features from RF to fit linear models and get their R2. 
# Enhance regularization.
# If R2 gets better then perhaps not over-fitting.

# Justin: Group the results by classes of cells (17).
# First PCA? 
# For each class use the 1th PCA. Then having 17 new "Cell Types". They're comparable.

In [27]:
# Eigenmodes reflect spectral clusters. 
# k-means on first 5 eigenmodes/ cell types - let's visualize that on the brain

In [ ]:
# Ask Justin and Harry for the visualization code.